In [1]:
# Radar signal simulator
# ECE 435/535 Winter 2020
# -------------------
# George P. Burdell
# Bell Georgina  
# Date: 
# -------------------
# Description of add ons 

'''
Method: -- this is a script..
1 - Create a pulse train from Transmitter
2 - Generate a list of targets, (Range, RCS)
3 - Generate return pulses for each of the targets into a single train
4 - Attenuate 1 to reasonable power level
5 - Add 3+4
6 - Add AGWN to 5
7 - Apply detection method
'''

from matplotlib import pyplot as plt
import numpy as np
import scipy.signal as sig

# Custom libraries
import pulsetrain as pt
import pulsenoise as pn
import detection as det

# Student Libary
import lastname as GPD # Intials (George P. Burdell)


ModuleNotFoundError: No module named 'pulsetrain'

## Student Custom Library 

In [ ]:
def myfuction(var1, var2='1942', var3='Chain Home'):
    print(var1, var2, var3)
    x = 42
    return x, var1

# usage
derp0, derp1 = myfuction(23,'1939')
print('\n================')
print(derp0, derp1)

def prob_detection(var1, var2)
    
    SNR = var1 #SNR obtained from the difference between noise floor power and peak power
    num_observations = var2
    
    false_detection = exp(-(SNR)*sqrt(num_observations))
    
    p = 1 - false_detection
    return p

def new_noise_floor(var1)
    
    sum_and_gated = var1
    
    noise floor = sum_and_gated; #find the average noise floor of the gated and summed signal
    
    return noise_floor



### Setup the RADAR parameters

In [ ]:
Pavg = 100e3        # Basic Power level output of the radar
Gt = 15             # Scalar Gain of TX antenna
Gr = Gt             # Scalar Gain of RX antenna  if Gr == Gt same antenna
fc = 40e6           # Carrier Frequency, Center Frequency
vp = 3e8            # Phase Velocity of the EM wave
NF = 1              # Receiver Noise Figure
T  = 1/fc           # period of one Carrier Frequency
#derived values
Lambda = vp/fc

# Setup Time portion
PRF = 500           # Pulses per second (hertz)
PRI = 1/PRF         # Pulse Repetition Interval (seconds)
R_unamb = PRI *vp/2 # Unambiguous Range

#Num cycles per pulse packet
k = 100             # k cycles of fc in the pulse packet
PW = k*T            # k cycles * Period of fc
BW = 1/PW           # Bandwidth of the RADAR Pulse
# error check
if PW >= PRI:
    print('Error: Pulse width much too long -- PRI: {}, PW = {}'.format(PRI, PW))


### Find R-max, R-umabigous

In [ ]:
# calculate maximum range with SNR = 1, n Observations = 1
SNRmin = 1
RCS = 1
Rmax = pt.calcRmax(Pavg,Gt,Gr,Lambda, BW, SNRmin = SNRmin, RCS = RCS) #, RCS, T, NF = 1,L=1, SNRmin=1)
print('Rmax(SNR:{}, RCS:{}) \t= {:.02f} km'.format(SNRmin, RCS, Rmax/1e3))
print('R unambigouse \t\t= {:.02f}km'.format(R_unamb/1e3))


# Setup the testing enviroment

In [ ]:
num_targets = 10

target_ranges = np.random.randint(Rmax//4,Rmax,num_targets)
target_rcs = np.random.randint(1,1000,num_targets)


#### Time series Constraints

In [ ]:
K_pulses = 20 # how many PRI's get simulated
dt_k = 20 # how many samples per fc period (Tc)

#### Build the sample Pulse Train and examine

In [ ]:
# Make a signal smaller in amplitude to simulate the 
# blanking / attenuation in normal RADAR systems


attenuate = True
if attenuate == True:
    dBm = -100 #dBm
    scalar = 1e-3 * np.power(10,(dBm/10))
else:
    scalar = Pavg
    
main_train, PW, dt, len_PRI = pt.createPulseTrain(A=scalar,fc = fc, k=k, PRI=PRI, dt_k=dt_k, K_pulses = K_pulses)



### Create the target reflections

In [ ]:
# Now we create the returns...
main_trace = np.zeros_like(main_train) # return without TX

for idx, target_range in enumerate(target_ranges):

    pwr, dbm = pt.RadarEquationAdv(Pavg, Gt, target_range, RCS, Gr, Lambda, dB=False)
    print(':: idx: {} Power at RX {} dBm @ range: {} rmax {}'.format(idx,(10*np.log10(Pavg/1e-3)),
                                                                     target_range, R_unamb ))
    p_train, PW, dt, len_PRI = pt.createPulseTrain(A=pwr,fc = fc, k=k, PRI=PRI,
                                                   dt_k=dt_k, K_pulses = np.int(K_pulses))
    # time shift to correct spot
    p_train = pt.timeShift(p_train, target_range,vp, dt, len_PRI)
    main_trace = main_trace + p_train

#### Merge the Transmitter Pulse train (main_train) with the reflected signals (main_trace)

#### add AWGN to the total Pulse train with returns

In [ ]:
# -------------------------------
# now we add the two systems together.
# Add noise to the pulse traing
main_trace = main_trace + main_train

main_trace = pn.addNoiseToPulseTrain(main_trace,1/PW)


## DETECTION method

In [ ]:
# -------------------------------
# Detection Section
# Envelope detect the signals
main_trace_env = det.envelope(main_trace)

# -------------------------------
# Gate the signal & sum them up for 
# provide n observation effects
n_obs_main_trace_env = main_trace_env.reshape(K_pulses+1, len_PRI)
# add them all together
n_obs_main_trace_env = n_obs_main_trace_env.sum(axis=0)


#### Threshold the data

In [ ]:
from scipy.signal import find_peaks

dBm = -80 #dBm
scalar = 1e-3 * np.power(10,(dBm/10))
#height = scalar

peaks, _ = find_peaks(n_obs_main_trace_env, height=scalar)


In [ ]:
# make the distance vector
nsamps = len(n_obs_main_trace_env)
x = np.linspace(0,R_unamb, nsamps)
dx = R_unamb/(len(n_obs_main_trace_env))


plt.semilogy(x/1e3,n_obs_main_trace_env)
plt.semilogy(peaks*dx/1e3,n_obs_main_trace_env[peaks], 'x')
plt.title('This is my title')
plt.xlabel('Distance in km')
plt.ylabel('Power in Watts')

# HW:
This homework is for getting familiar with interacting with data. You are going to write a few functions. The functions that you are writing are relively simple. The basic simulator is supplied. I suggest you exercise it change a few functions. You have two method of adding funtions. You can either add them near the top in the #Custom Fuction# block or in the <i><b>LastName.py</b></i> file 


## Jupyter Modifications 
1. Add probablity of detection function
1. Add fuction to detect/generate new noise floor after gating and summing
1. Use probability of detection function to determine threshold
1. What is last peak? 
1. Convert plot to dBm
1. Replace find_peaks with a thresholding function

## Functions to add
1. Probablity of False Positive 
1. Find noise floor from data
1. SNR Threshold function
1. Report number of targets and distance